# Estimates of the sensitivity
In this notebook, we estimates the sensitivities for LETO (Line Emission Terahertz Observatory) and save the data to be plotted and analysed later on. First, we import the Python packages

In [1]:
import numpy as np

import astropy.units as u
from astropy.table import Table
from astropy.cosmology import Planck13, FlatLambdaCDM

from telescope_estimator import LuminEstimator

cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

## Reading and pre-processing data

We read the pySIDES original data

In [2]:
# This file is very large, so we recommend downloading the file yourself
cat = Table.read('pySIDES_from_original.fits', format='fits')

We also read the coefficients calculated for the paper (It is possible to also select the case where only HII regions are taken into account)

In [3]:
# Change the file that you want to use manually or uncomment the respective lines in the notebook
# estimated_linreg = Table.read('../data/raw//coeff_linreg.csv', format='ascii.csv')
# estimated_linreg = Table.read('../data/raw//coeff_linreg_HII.csv', format='ascii.csv') # In case HII
estimated_linreg = Table.read('../data/interim/coeff_MLPreg.csv',
                              format='ascii.csv')

# Dictionaries to create the analog of the lines names
linreg_lines = {'CII158': 'logLCII', 'NIII': 'logLNIII_57',
                'NII205': 'logLNII_205', 'NII122': 'logLNII_122',
                'OI145': 'logLOI_145',  'OI63': 'logLOI_63',
                'OIII88': 'logLOIII_88', 'OIII52': 'logLOIII_51'}
inv_linreg_lines = {v: k for k, v in linreg_lines.items()}

We calculate the log(1+z) and log(SFR) from the SIDES catalog. With this we can estimate the expected line luminosity for the galaxies in the catalog.

In [4]:
logz = np.log10(1 + cat['redshift'])
logsfr = np.log10(cat['SFR'])

# for line in linreg_lines:
#     row = np.where(estimated_linreg['line'] == linreg_lines[line])[0]
#     cat['L'+line] = (estimated_linreg['Intercept'][row] +
#                      estimated_linreg['cSFR'][row]*logsfr +
#                      estimated_linreg['clogz'][row]*logz + estimated_linreg['cSFRlogz'][row]*logz*logsfr)

for line in linreg_lines:
    row = np.where(estimated_linreg['line'] == linreg_lines[line])[0]
    cat['L'+line] = (estimated_linreg['Intercept'][row] +
                     estimated_linreg['cSFR'][row]*logsfr +
                     estimated_linreg['cSFR2'][row]*logsfr*logsfr +
                     estimated_linreg['clogz'][row]*logz +
                     estimated_linreg['cSFRlogz'][row]*logz*logsfr)

<ipython-input-4-244383a5b828>:2: RuntimeWarning: divide by zero encountered in log10
  logsfr = np.log10(cat['SFR'])
<ipython-input-4-244383a5b828>:12: RuntimeWarning: invalid value encountered in add
  cat['L'+line] = (estimated_linreg['Intercept'][row] +


## Using the estimator to calculate the bands sensitivies
We redefine the class for the LETO estimator to start our calculations

In [5]:
LETO_estimator = LuminEstimator()

We run a simple verification to verify is working

In [6]:
# Inputs
HTIME = 1  # In hours
# bw = 18.75 * u.MHz # bandwidth
line_width = 12 * u.km/u.s
int_time = 3600*HTIME * u.s
NPOL = 1.4  # 2 for polarised measurement

list_int_time = [int_time, int_time*10, int_time*100, int_time*1000]

LETO_estimator.estimate_flux(line_width, list_int_time, NPOL)

# Print as an example
LETO_estimator.retrive_lum(Planck13.luminosity_distance(0.2))

[array([6.20381168, 7.22619492, 7.81754341, 7.86823446, 7.9015483 ,
        7.95230346, 7.04639136, 7.1645728 ]),
 array([5.70381168, 6.72619492, 7.31754341, 7.36823446, 7.4015483 ,
        7.45230346, 6.54639136, 6.6645728 ]),
 array([5.20381168, 6.22619492, 6.81754341, 6.86823446, 6.9015483 ,
        6.95230346, 6.04639136, 6.1645728 ]),
 array([4.70381168, 5.72619492, 6.31754341, 6.36823446, 6.4015483 ,
        6.45230346, 5.54639136, 5.6645728 ])]

In [8]:
LETO_estimator.table

Species,Wavelength,Frequency,Quantum Noise,Expected DSB TN,QN,G_FR,T_RF,G_Mix,G_mix,T_IF,Expected Tsys,Beam solid angle,Range,BW,Sensitivity DT,Channel,Min Bright,Min Bright2,Flux Density,Flux 1h,Flux 10h,Flux 100h,Flux 1000h
,micron,GHz,K,K,K,,K,dbit,dbit,K,K,sr,GHz,MHz,K,km / s,W / (Hz m2 sr),W / (m2 sr),W / (Hz m2),W / m2,W / m2,W / m2,W / m2
str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
CI [608],609.3,492.0276678155261,23.613603766681994,40.0,1.6939388157447897,0.86,50.0,-1.0,0.7943282347242815,5.0,65.06902140920859,1.477142849725397e-07,8.0,18.0,8.002023274441711e-06,10.9674,5.951832935489668e-22,1.172197525453406e-14,8.791707463418682e-29,5.004327619134813e-20,1.582507343415363e-20,5.004327619134812e-21,1.5825073434153628e-21
CI [370],370.6,808.9380949811117,38.82290549120167,130.0,3.3485386617820647,0.86,50.0,-2.0,0.6309573444801932,5.0,176.9426619307219,5.464758449948197e-08,16.0,269.85,5.61995881325441e-06,100.00641000000002,1.1298884811230528e-21,3.65857036521226e-14,6.174567624716336e-29,5.2690094058375135e-19,1.6662070735297034e-19,5.269009405837513e-20,1.6662070735297034e-20
NII [205],205.3,1460.2652605942524,70.08167937184285,400.0,5.707625781592078,0.86,50.0,-4.9,0.3235936569296282,5.0,515.7837802566238,1.677019216345521e-08,16.0,483.67,1.2236434587358577e-05,99.29745100000002,8.01660107331176e-21,4.685787281039509e-13,1.344399404971995e-28,2.056257324880769e-18,6.502456602028048e-19,2.056257324880769e-19,6.502456602028049e-20
CII [158],157.68,1901.2712962962964,91.24663099340016,390.0,4.27413040628545,0.86,50.0,-5.5,0.28183829312644537,5.0,506.369422017166,9.892666372416988e-09,16.0,633.77,1.0494540328915274e-05,99.9328536,1.1655300540756161e-20,8.870091735735283e-13,1.15301999719952e-28,2.3108327672473593e-18,7.307494836251401e-19,2.3108327672473595e-19,7.307494836251398e-20
OI [145],145.53,2060.0045214045217,98.86462430453747,400.0,4.045936580589845,0.86,50.0,-6.5,0.22387211385683395,5.0,524.1870797534019,8.426848749709596e-09,16.0,689.48,1.04156712349266e-05,100.3400244,1.35798659460807e-20,1.1197580660548918e-12,1.1443547636895407e-28,2.495067819018311e-18,7.890097224686647e-19,2.495067819018311e-19,7.890097224686645e-20
NII [122],121.8,2461.3502298850576,118.12618041904219,410.0,3.470864786667623,0.86,50.0,-7.2,0.19054607179632474,5.0,541.1655717012943,5.902754444886524e-09,16.0,817.23,9.876880436874497e-06,99.53861400000001,1.8383935950197423e-20,1.8112252167682193e-12,1.08515859644537e-28,2.8043842289479085e-18,8.868241597730497e-19,2.8043842289479085e-19,8.868241597730498e-20
OH [119],119.0,2519.2643529411766,120.90561995831376,344.0,2.845194459270011,0.86,50.0,-6.0,0.251188643150958,5.0,452.8495656442542,5.634482872810825e-09,16.0,18.0,5.56902914109803e-05,2.142,1.0859225357853054e-19,1.0950479352456863e-11,6.118611929081603e-28,3.4827749666234118e-19,1.101350147234689e-19,3.482774966623412e-20,1.1013501472346885e-20
OI [63],63.18,4745.053149730928,227.7266346160073,460.0,2.019965739956822,0.86,50.0,-6.0,0.251188643150958,5.0,594.4774726209984,1.5882518996530322e-09,8.0,18.0,7.310733232221775e-05,1.13724,5.05725639268689e-19,9.605425247268549e-11,8.032197072717394e-28,4.572006725722331e-19,1.445795473089131e-19,4.572006725722333e-20,1.4457954730891308e-20


Create dictionaries to work with the estimators

In [10]:
list_lines = [inv_linreg_lines[line] for line in estimated_linreg['line'].data]
disp = {}
for A, B in zip(list_lines, estimated_linreg['sigma'].data):
    disp[A] = B

wave_lines = {'OIII52': 51.81, 'NIII': 57.34, 'OI63': 63.18,
              'OIII88': 88.36, 'NII122': 121.8, 'OI145': 145.53,
              'CII158': 157.68, 'NII205': 205.3}
# wave_lines = {'OIII88': 88.36,
#               'CII158': 157.68}

Finally, we estimate the lines that we could observe with the configuration of the telescope

In [11]:
total_array = []
bands_to_use = [False, True, True, True, True, True, False, False]
# bands_to_use = [False, True, True, False, False, False, False, False]
for line in wave_lines:
    for irow, row in enumerate(LETO_estimator.table[bands_to_use]):
        redsh = (row['Wavelength']-wave_lines[line])/wave_lines[line]
        freq = (wave_lines[line] * u.micron).to(u.GHz,
                                                equivalencies=u.spectral()).value
        if redsh <= -0.05:
            print('Local Universe')
        else:
            lum_min = np.array(LETO_estimator.retrive_lum(
                Planck13.luminosity_distance(redsh))).T[bands_to_use][irow]
            lum_min = np.log10(10**lum_min / (1+redsh))
            flux_min = []
            for col in LETO_estimator.table.colnames:
                if col.startswith('Flux'):
                    if col.endswith('h') or col.endswith('x'):
                        flux_min.append(
                            LETO_estimator.table[bands_to_use][col][irow])
            flux_min = np.array(flux_min)
            band_z = (row['Frequency'] + row['Range'],
                      row['Frequency'] - row['Range'])
            redsh1 = ((freq - band_z[0]) / band_z[0])
            redsh2 = ((freq - band_z[1]) / band_z[1])
            secl = cat[np.logical_and.reduce(
                [cat['redshift'] > redsh1, cat['redshift'] < redsh2, cat['SFR'] > 0])]
            print(redsh, line, row['Species'],
                  lum_min, flux_min, redsh1, redsh2)
            allcounts = []
            for iterat in range(500):
                # Because SIDEs is 2deg^2
                sample = (secl.to_pandas().sample(frac=0.5))
                rand_lum = disp[line] * \
                    (2*np.random.random(size=len(sample)) - 1)
                counts, bins = np.histogram(sample['L'+line] + rand_lum,
                                            bins=np.linspace(5, 12, 71))
                allcounts.append(counts)
#             print('Estimated', np.median(allcounts, axis=0),
#                   np.std(allcounts, axis=0),
#                   np.percentile(allcounts, 75, axis=0))
            counts = np.mean(allcounts, axis=0)  # maybe median?
            er_counts = np.std(allcounts, axis=0)
            poisson_error = np.sqrt(counts)
            per25_counts = np.percentile(allcounts, 25, axis=0)
            per75_counts = np.percentile(allcounts, 75, axis=0)
            total_array.append([line, row['Species'], redsh, redsh1, redsh2, list_int_time,
                                lum_min, flux_min, bins, counts, er_counts, poisson_error,
                                per25_counts, per75_counts])
#             print(total_array)

6.153059254970083 OIII52 CI [370] [9.92994282 9.42994282 8.92994282 8.42994282] [5.26900941e-19 1.66620707e-19 5.26900941e-20 1.66620707e-20] 6.014322847019202 6.2973945426853835
2.9625554912179117 OIII52 NII [205] [10.02812602  9.52812602  9.02812602  8.52812602] [2.05625732e-18 6.50245660e-19 2.05625732e-19 6.50245660e-20] 2.919608678370764 3.0064538591904264
2.0434279096699477 OIII52 CII [158] [9.80269684 9.30269684 8.80269684 8.30269684] [2.31083277e-18 7.30749484e-19 2.31083277e-19 7.30749484e-20] 2.018029914796303 2.069257002093082
1.8089171974522291 OIII52 OI [145] [9.74137207 9.24137207 8.74137207 8.24137207] [2.49506782e-18 7.89009722e-19 2.49506782e-19 7.89009722e-20] 1.7872685571454006 1.8309047589711014
1.3508975101331788 OIII52 NII [122] [9.55761694 9.05761694 8.55761694 8.05761694] [2.80438423e-18 8.86824160e-19 2.80438423e-19 8.86824160e-20] 1.3357142067357113 1.3662795031509638
5.463201953261248 NIII CI [370] [9.85379567 9.35379567 8.85379567 8.35379567] [5.26900941e-19

We organize the table and we check the results

In [12]:
All = Table(np.array(total_array), names=['Line', 'Band', 'z', 'z_min', 'z_max',
                                          'Int_times',
                                          'Lums_min', 'Fluxes_min',
                                          'Lum_bins', 'mean_counts',
                                          'std', 'poisson_err', 'p25', 'p75'])

<ipython-input-12-80d6d5ad6f74>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  All = Table(np.array(total_array), names=['Line', 'Band', 'z', 'z_min', 'z_max',


In [13]:
All[-2]

Line,Band,z,z_min,z_max,Int_times,Lums_min,Fluxes_min,Lum_bins,mean_counts,std,poisson_err,p25,p75
object,object,object,object,object,object,object,object,object,object,object,object,object,object
NII205,CI [370],0.8051631758402338,0.770151323448928,0.8415879749465648,"[<Quantity 3600. s>, <Quantity 36000. s>, <Quantity 360000. s>, <Quantity 3600000. s>]",[8.3923629 7.8923629 7.3923629 6.8923629],[5.26900941e-19 1.66620707e-19 5.26900941e-20 1.66620707e-20],[ 5. 5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6. 6.1 6.2 6.3 6.4 6.5 6.6 6.7 6.8 6.9 7. 7.1 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8. 8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9. 9.1 9.2 9.3 9.4 9.5 9.6 9.7 9.8 9.9 10. 10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11. 11.1 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12. ],[3.03064e+02 2.76538e+02 2.53340e+02 2.28240e+02 2.05992e+02 1.86470e+02 1.68344e+02 1.50916e+02 1.36090e+02 1.21332e+02 1.09054e+02 9.76560e+01 8.69960e+01 7.84940e+01 6.83700e+01 5.99840e+01 5.20220e+01 4.42640e+01 3.85320e+01 3.12800e+01 2.49200e+01 1.89180e+01 1.48740e+01 1.10800e+01 7.63200e+00 5.22400e+00 3.33800e+00 2.31200e+00 1.41200e+00 1.01800e+00 6.64000e-01 3.02000e-01 2.16000e-01 1.66000e-01 1.22000e-01 4.40000e-02 1.60000e-02 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00],[17.04957196 16.55706967 15.70809982 14.72597705 13.87371385 13.42211235 12.50094652 12.21838549 11.14477007 10.4039308 10.50271793 9.81986069 9.25310672 8.82734184 8.36570977 7.34409586 7.47887131 6.60471831 5.99407841 5.76451212 4.9716798 4.41217361 3.93371631 3.41666504 2.70639539 2.39954662 1.83949884 1.48548174 1.11635837 0.98472128 0.77659771 0.53553338 0.4704721 0.39299364 0.34513186 0.21462525 0.1254751 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ],[17.40873344 16.62943174 15.91665794 15.10761397 14.3524214 13.65540186 12.9747447 12.28478734 11.66576187 11.01508057 10.44289232 9.88210504 9.32716463 8.85968397 8.26861536 7.74493383 7.21262782 6.65311957 6.20741492 5.59285258 4.99199359 4.34948273 3.85668251 3.3286634 2.76260746 2.28560714 1.82701943 1.52052622 1.18827606 1.00895986 0.81486195 0.54954527 0.464758 0.40743098 0.34928498 0.20976177 0.12649111 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ],[291. 265. 243. 218. 197. 178.75 160. 142.75 128. 114. 101. 91. 81. 72. 62.75 55. 47. 40. 35. 27. 22. 16. 12. 9. 6. 4. 2. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ],[314. 287.25 264. 238. 216. 194. 176. 159. 143. 128. 115. 104. 93. 84. 74. 65. 57. 48. 43. 36. 28. 22. 17. 13. 9. 7. 4. 3. 2. 2. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ]


Finally, we save the file to further analysis

In [14]:
# np.save('../data/processed/NcountsMay_SIDES_LETO_percentiles_poisson_HII', All.as_array())
# np.save('../data/processed/NcountsMay_SIDES_LETO_percentiles_poisson', All.as_array())
np.save('../data/processed/NcountsMay_SIDES_LETO_percentiles_poisson_MLP', All.as_array())

##### Notebook information

In [15]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w 

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.59.1.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy : 5.0
numpy   : 1.22.1
autopep8: 1.5.7
json    : 2.0.9

Watermark: 2.2.0

